# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an quivilent file their allocation package

In [1]:
import pandas as pd
import os

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

In [2]:
file_list = [
    "sierra_fund_request.xlsx",
    "slocog_fund_request.xlsx",
    "tehema_fund_request.xlsx",
    "tuolumne_fund_request.xlsx",
    "ventura_fund_request.xlsx",
    "alpine_fund_request.xlsx",
    "amador_fund_request.xlsx",
    "butte_fund_request.xlsx",
    "calaveras_fund_request.xlsx",
    "del_norte_fund_request.xlsx",
    "el_dorado_fund_request.xlsx",
    "humboldt_fund_request.xlsx",
    "kern_fund_request.xlsx",
    "kings_fund_request.xlsx",
    "la_metro_fund_request.xlsx",
    "lake_fund_request.xlsx",
    "lassen_fund_request.xlsx",
    "madera_fund_request.xlsx",
    "mariposa_fund_request.xlsx",
    "mendocino_fund_request.xlsx",
    "merced_fund_request.xlsx",
    "mtc_fund_request.xlsx",
    "nevada_fund_request.xlsx",
    "orange_fund_request.xlsx",
    "placer_fund_request.xlsx",
    "plumas_fund_request.xlsx",
    "riverside_fund_request.xlsx",
    "san_benito_fund_request.xlsx",
    "san_diego_mts_fund_request.xlsx",
    "santa_cruz_fund_request.xlsx",
    "shasta_fund_request.xlsx",
]

In [ ]:
# loop to extract basename of files
file_name = []

for file in file_list:
    name_extract = os.path.splitext(os.path.basename(file))[0]
    file_name.append(name_extract)

In [ ]:
file_name

In [3]:
def data(file:str) -> pd.DataFrame:
    col_names = [
    "RTPA",
    "Implementing Agenc-y/-ies",
    "Project",
    "Fund Source",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "total",
]
    df = pd.read_excel(f"{GCS_PATH}{file}", header=2, nrows=40, names=col_names).drop(columns="total")
    df = df.dropna(how= "all")
    return df


In [19]:
def read_in(file:str) -> pd.DataFrame:
    df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
    df = df.dropna(how= "all")
    return df

In [4]:
alpine = data("alpine_fund_request.xlsx")

In [5]:
amador = data("amador_fund_request.xlsx")

In [6]:
display(alpine.dtypes, alpine)

RTPA                          object
Implementing Agenc-y/-ies     object
Project                       object
Fund Source                   object
capital_FY23-24              float64
capital_FY24-25              float64
capital_FY25-26              float64
capital_FY26-27              float64
operating_FY23-24            float64
operating_FY24-25            float64
operating_FY25-26            float64
operating_FY26-27            float64
dtype: object

,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,360641.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,NaN,360794.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (GGRF),3616.684,3781.0,3781.0,3781.0,NaN,NaN,NaN,NaN
3,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (PTA),3123.316,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Grand Total,NaN,NaN,NaN,367381.000,364575.0,3781.0,3781.0,0.0,0.0,0.0,0.0


In [7]:
display(
    amador.columns,
    amador.head()
)

Index(['RTPA', 'Implementing Agenc-y/-ies', 'Project', 'Fund Source',
       'capital_FY23-24', 'capital_FY24-25', 'capital_FY25-26',
       'capital_FY26-27', 'operating_FY23-24', 'operating_FY24-25',
       'operating_FY25-26', 'operating_FY26-27'],
      dtype='object')

,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
2,Amador County Transportation Commission,Amador County Transportation Commission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Amador Transit,NaN,TIRCP,100000.0,75000.0,NaN,NaN,NaN,NaN,NaN,NaN
20,Grand Total,NaN,NaN,NaN,100000.0,75000.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
file_list.sort()

In [11]:
file_list

['alpine_fund_request.xlsx',
 'amador_fund_request.xlsx',
 'butte_fund_request.xlsx',
 'calaveras_fund_request.xlsx',
 'del_norte_fund_request.xlsx',
 'el_dorado_fund_request.xlsx',
 'humboldt_fund_request.xlsx',
 'kern_fund_request.xlsx',
 'kings_fund_request.xlsx',
 'la_metro_fund_request.xlsx',
 'lake_fund_request.xlsx',
 'lassen_fund_request.xlsx',
 'madera_fund_request.xlsx',
 'mariposa_fund_request.xlsx',
 'mendocino_fund_request.xlsx',
 'merced_fund_request.xlsx',
 'mtc_fund_request.xlsx',
 'nevada_fund_request.xlsx',
 'orange_fund_request.xlsx',
 'placer_fund_request.xlsx',
 'plumas_fund_request.xlsx',
 'riverside_fund_request.xlsx',
 'san_benito_fund_request.xlsx',
 'san_diego_mts_fund_request.xlsx',
 'santa_cruz_fund_request.xlsx',
 'shasta_fund_request.xlsx',
 'sierra_fund_request.xlsx',
 'slocog_fund_request.xlsx',
 'tehema_fund_request.xlsx',
 'tuolumne_fund_request.xlsx',
 'ventura_fund_request.xlsx']

In [20]:
butte = read_in('butte_fund_request.xlsx')
butte

,SB125 Fund Request Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,The following template is designed to capture ...,NaN,NaN,NaN,Project Development,NaN,Capital,NaN,NaN,NaN,Total
1,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY23-24,FY24-25,FY25-26,FY26-27,NaN
2,Butte County Association of Governments,Butte County Association of Governments and th...,North Valley Rail,TIRCP,10488545,4511455,NaN,NaN,NaN,NaN,15000000
3,Butte County Association of Governments,Butte Regional Transit,Butte Regional Transit Zero-Emission Deployment,TIRCP,NaN,200000,NaN,5802814,NaN,NaN,6002814
4,NaN,NaN,NaN,ZETCP (PTA),200000,NaN,315836,NaN,NaN,NaN,515836
5,NaN,NaN,NaN,ZETCP (GGRF),NaN,NaN,597319.51,NaN,NaN,NaN,597319.51
6,NaN,NaN,NaN,ZETCP (GGRF),NaN,200000,NaN,1673362,NaN,NaN,1873362
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [21]:
read_in('calaveras_fund_request.xlsx')

,SB125 Fund Request Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,The following template is designed to capture ...,NaN,NaN,NaN,Capital,NaN,NaN,NaN,Operating,NaN,NaN,NaN,Total
1,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,FY23-24,FY24-25,FY25-26,FY26-27,NaN
2,Calaveras County Council of Goverments,Calaveras Transit Angency,CTA Sustainable Transit Infrastructure and Tec...,TIRCP,2576611,2582360,NaN,NaN,NaN,NaN,NaN,NaN,5158971
3,Calaveras County Council of Goverments,Calaveras Transit Agency,CTA Sustainable Transit Infrastructure and Tec...,ZETCP (GGRF),238532,133811,133811,133811,NaN,NaN,NaN,NaN,639965
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [22]:
read_in('del_norte_fund_request.xlsx')

,SB125 Fund Request Template,Unnamed: 1,DNLTC Draft,2023-12-27 00:00:00,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,The following template is designed to capture ...,NaN,NaN,NaN,Capital,NaN,NaN,NaN,Operating,NaN,NaN,NaN,Total
1,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,FY23-24,FY24-25,FY25-26,FY26-27,NaN
2,DNLTC,Redwood Coast Transit,Transit Center,TIRCP,500000,1556248,NaN,NaN,NaN,NaN,NaN,NaN,2056248
3,DNLTC,Redwood Coast Transit,Electric Buses,TIRCP,1175501,122726,NaN,NaN,NaN,NaN,NaN,NaN,1298227
4,DNLTC,Redwood Coast Transit,Electric Buses,ZETCP (PTA),69395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69395
5,DNLTC,Redwood Coast Transit,Electric Buses,ZETCP (GGRF),80357,84007,84007,84007,NaN,NaN,NaN,NaN,332378
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [23]:
read_in('el_dorado_fund_request.xlsx')

,SB125 Fund Request Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,The following template is designed to capture ...,NaN,NaN,NaN,Capital,NaN,NaN,NaN,Operating,NaN,NaN,NaN,Total
1,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,FY23-24,FY24-25,FY25-26,FY26-27,NaN
2,El Dorado County Transportation Commission,El Doado County Transit Authority,ZEB Fleet Conversion,TIRCP,9085857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9085857
3,El Dorado County Transportation Commission,El Doado County Transit Authority,ZEB Fleet Conversion,ZETCP (PTA),242312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,242312
4,El Dorado County Transportation Commission,El Doado County Transit Authority,ZEB Fleet Conversion,ZETCP (GGRF),521458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,521458
5,El Dorado County Transportation Commission,El Dorado County Transportation Commission,Administration of SB 125 Funding,ZETCP (PTA),NaN,NaN,NaN,NaN,208011,NaN,NaN,NaN,208011
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [25]:
read_in("humboldt_fund_request.xlsx")

,SB125 Fund Request Template,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,The following template is designed to capture ...,NaN,NaN,NaN,Capital,NaN,NaN,NaN,Operating,NaN,NaN,NaN,Total
1,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,FY23-24,FY24-25,FY25-26,FY26-27,FY23-24,FY24-25,FY25-26,FY26-27,NaN
2,Humboldt County Association of Governments,Humboldt Transit Authority,Maintain Existing Service,TIRCP,NaN,NaN,NaN,NaN,772834,1910431.32,2971955.64,7415199,13070419.96
3,Humboldt County Association of Governments,Humboldt Transit Authority,Maintain Existing Service,ZETCP (PTA),NaN,NaN,NaN,NaN,342326,0,0,0,342326
4,Humboldt County Association of Governments,Humboldt Transit Authority,Maintain Existing Service,ZETCP (GGRF),NaN,NaN,NaN,NaN,391693,432746.99,432746.99,432747,1689933.98
5,Humboldt County Association of Governments,Arcata and Mad River Transit System,Operations,TIRCP,NaN,NaN,NaN,NaN,204188,413152,291131,-,908471
6,Humboldt County Association of Governments,Arcata and Mad River Transit System,Operations,ZETCP (GGRF),NaN,NaN,NaN,NaN,51028.06,30084.02,30084.02,30084.02,141280.12
7,Humboldt County Association of Governments,Yurok Tribal Transportation System,Operations,TIRCP,NaN,NaN,NaN,NaN,150000,150000,-,-,300000
8,Humboldt County Association of Governments,HCAOG,RTPA Administration,ZETCP (PTA),NaN,NaN,NaN,NaN,40000,-,-,-,40000
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
